In [ ]:
import os
import sys
from collections import OrderedDict
from pathlib import Path

import numpy as np
from threeML import FermiGBMBurstCatalog, download_GBM_trigger_data, TimeSeriesBuilder, update_logging_level
import yaml

parent_dir = os.path.abspath("..")
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from real_data import AnalysisBuilder, SynchSample
from zusammen import DataSet

update_logging_level("FATAL")

In [ ]:
data_folder = "real_data/"

## Own Sample

In [ ]:
grb_names = [
    "GRB160509374",
    "GRB141028455",
    "GRB140808038",
    "GRB140606133",
    "GRB131011741",
    "GRB130518580",
    "GRB120811649",
    "GRB120729456",
    "GRB120326056",
    "GRB120119170",
    "GRB110213220",
    "GRB100816026",
    "GRB100728439",
    "GRB091020900",
    "GRB090926914",
    "GRB081222204",
    "GRB081221681",
    "GRB081118876"
]
data_paths = {grb: data_folder + grb for grb in grb_names}
len(grb_names)

In [ ]:
z = dict(
    GRB160509374=1.17,
    GRB141028455=2.33,
    GRB140808038=3.29,
    GRB140606133=0.384,
    GRB131011741=1.874,
    GRB130518580=2.488,
    GRB120811649=2.671,
    GRB120729456=0.8,
    GRB120326056=1.798,
    GRB120119170=1.728,
    GRB110213220=1.46,
    GRB100816026=0.8049,
    GRB100728439=2.106,
    GRB091020900=1.71,
    GRB090926914=1.24,
    GRB081222204=2.77,
    GRB081221681=2.26,
    GRB081118876=2.58
)
source_time = dict(
    GRB160509374=[9, 24],
    GRB141028455=[6, 22],
    GRB140808038=[0, 5],
    GRB140606133=[0, 8],
    GRB131011741=[0, 8],
    GRB130518580=[23, 32],
    GRB120811649=[-2, 9],
    GRB120729456=[0, 8],
    GRB120326056=[0, 8],
    GRB120119170=[7, 23],
    GRB110213220=[13, 23],
    GRB100816026=[0, 5],
    GRB100728439=[0, 6],
    GRB091020900=[0, 10],
    GRB090926914=[0, 40],
    GRB081222204=[0, 14],
    GRB081221681=[17, 34],
    GRB081118876=[5, 13]
)

for i in grb_names:
    assert i in z
    assert i in source_time

for i in z:
    assert i in grb_names

for i in source_time:
    assert i in grb_names

### Download GRBs from Catalog

In [ ]:
det_info = dict()
gbm_catalog = FermiGBMBurstCatalog()
for grb in grb_names:
    gbm_catalog.query_sources(grb)
    det_info[grb] = gbm_catalog.get_detector_information()[grb]
grb_names[-1], det_info[grb_names[-1]]

In [ ]:
dload_dict = dict()
for grb, info in det_info.items():
    dload = download_GBM_trigger_data(info["trigger"], info["detectors"], data_paths[grb], compress_tte=False)

    dload_dict[grb] = OrderedDict()
    dload_dict[grb]["z"] = z[grb]
    dload_dict[grb]["source_time"] = source_time[grb]
    for det, files in dload.items():
        dload_dict[grb][str(det)] = OrderedDict()
        for kind, path in files.items():
            dload_dict[grb][str(det)][str(kind)] = str(path.absolute())

In [ ]:
with open(data_folder + "dload.yml", "r") as f:
    dload = yaml.load(f, Loader=yaml.SafeLoader)

In [ ]:
file_name = data_folder + "dload.yml"
with open(file_name, "w") as f:
    yaml.dump(dload_dict, stream=f, default_flow_style=False)

In [ ]:
dload_dict[grb_names[0]]

In [ ]:
%matplotlib widget
grb = grb_names[0]
detector = det_info[grb]["detectors"][1]
ts_cspec = TimeSeriesBuilder.from_gbm_cspec_or_ctime(
        detector, cspec_or_ctime_file=dload_dict[grb][detector]["cspec"], rsp_file=dload_dict[grb][detector]["rsp"]
    )

print(grb)
ts_cspec.view_lightcurve(-10,60)

### Analysis

In [ ]:
ab = AnalysisBuilder("dload.yml", data_folder, 5, 4)
ab.write_yaml(data_folder + "data.yml")

ds = DataSet.from_yaml(data_folder + "data.yml")
ds.to_hdf5_file(data_folder + "data.h5")

## Synch Sample

In [ ]:
base_path = Path(data_folder)
grb_names = [x.name for x in base_path.glob("GRB*")]

In [ ]:
ss = SynchSample(base_path=base_path, grb_names=grb_names, sig_min=5, interval_min=4)
ss.write_yaml(data_folder + "data.yml")

In [11]:
ds = DataSet.from_yaml(data_folder + "data.yml")
ds.to_hdf5_file(data_folder + "data.h5")